In [ ]:
import csv
import os
from dotenv import load_dotenv
import pandas as pd
import json
from datetime import datetime
import numpy as np

load_dotenv()

csp_filepath = os.environ['CSP_PATHFILE']
base_output_filepath = os.environ['DB_EXPORT']
new_generated_codes_filepath = os.environ['NEW_ID_PSP']


In [ ]:
csp_column_type = {
    'adresse-allocataire_voie': 'str',
    'adresse-allocataire_code_insee': 'str'
}


In [ ]:
csp_df = pd.read_csv(csp_filepath, on_bad_lines='skip', sep=',', engine="c", dtype=csp_column_type)

In [ ]:
new_generated_codes = pd.read_csv(new_generated_codes_filepath, on_bad_lines='skip', sep=',', engine="c")

In [ ]:
df_final_with_new_code = csp_df.reset_index(drop=True).combine_first(new_generated_codes.reset_index(drop=True))

In [ ]:
df_final_with_new_code['allocataire-telephone'] = df_final_with_new_code['allocataire-telephone'].str.replace(' ', '')

In [ ]:
# assign code_orgnisme for csp
df_final_with_new_code['allocataire-code_organisme'] = '459'
df_final_with_new_code['organisme'] = 'MSA'

In [ ]:
df_final_with_new_code['adresse-allocataire_code_insee'] = df_final_with_new_code['adresse-allocataire_code_insee'].astype(str)

In [ ]:
# map to json values for target DB model
## map allocataire json
def to_json_allocataire_without_null(row):
    allocataire_mapping = {
        'qualite': row['allocataire-qualite'],
        'matricule': row['allocataire-matricule'],
        'code_organisme': row['allocataire-code_organisme'],
        'telephone': row['allocataire-telephone'],
        'nom': row['allocataire-nom'],
        'prenom': row['allocataire-prenom'],
        'date_naissance': row['allocataire-date_naissance'],
        'courriel': row['allocataire-courriel'],
        'code_insee_commune_naissance': row['allocataire-code_insee_commune_naissance'],
        'commune_naissance': row['allocataire-commune_naissance'],
        'code_iso_pays_naissance': row['allocataire-code_iso_pays_naissance'],
    }
    filtered_NaN_allocataire = {k: v for k, v in allocataire_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_NaN_allocataire, ensure_ascii=False)

df_final_with_new_code['allocataire'] = df_final_with_new_code.apply(to_json_allocataire_without_null, axis=1)

In [ ]:
## map adresse_allocataire json
def to_json_adresse_without_null(row):
    adresse_mapping = {
        'voie': row['adresse-allocataire_voie'],
        'code_postal': row['adresse-allocataire_code_postal'],
        'nom_adresse_postale': row['adresse-allocataire_nom_adresse_postal'],
        'commune': row['adresse-allocataire_commune'],
        'code_insee': row['adresse-allocataire_code_insee'],
        'cplt_adresse': row['adresse-allocataire_cptl_adresse'],
    }
    
    filtered_address = {k: v for k, v in adresse_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_address, ensure_ascii=False)

df_final_with_new_code['adresse_allocataire'] = df_final_with_new_code.apply(to_json_adresse_without_null, axis=1)

In [ ]:
df_final_with_new_code['date_naissance'] = pd.to_datetime(df_final_with_new_code['date_naissance']).dt.floor('D') + pd.DateOffset(hours=4)

In [ ]:
# Add missing default column needed for target DB model
import datetime

df_final_with_new_code['updated_at'] = datetime.datetime.now()
df_final_with_new_code['exercice_id'] = 3
df_final_with_new_code['uuid_doc'] = np.NaN
df_final_with_new_code[['zrr', 'qpv', 'a_valider', 'refuser']] = False
df_final_with_new_code['id'] = np.NaN
timestamp_with_custom_tz = pd.Timestamp.now(tz='Europe/Paris')
df_final_with_new_code["created_at"] = timestamp_with_custom_tz

In [ ]:
df_final_with_new_code_column_filtered = df_final_with_new_code[['id', 
       'date_naissance',
       'genre', 'id_psp', 'nom', 'organisme', 'prenom',
       'allocataire', 'adresse_allocataire',
       'updated_at', 'exercice_id', 'uuid_doc', 'zrr', 'qpv', 'a_valider', 'refuser', 'created_at']]

In [ ]:
# output to CSV
df_final_with_new_code_column_filtered.to_csv(base_output_filepath)

In [ ]:
df_campaign=df_final_with_new_code.copy()

In [ ]:
# Ajout d'une colonne pour le sexe 
df_campaign['neele'] = 'Né le'
mask_girl = df_campaign['genre'] == 'F'
df_campaign.loc[mask_girl, 'neele'] =  'Née le'

In [ ]:
# capitalize on name / surname
df_campaign['allocataire-prenom'] = df_campaign['allocataire-prenom'].astype(str).str.capitalize()
df_campaign['allocataire-nom'] = df_campaign['allocataire-nom'].astype(str).str.capitalize()
df_campaign['beneficiaire-prenom'] = df_campaign['prenom'].astype(str).str.capitalize()
df_campaign['beneficiaire-nom'] = df_campaign['nom'].astype(str).str.capitalize()

In [ ]:
df_campaign.rename(columns={
    'allocataire-courriel' : 'email',
    'id_psp': 'code',
    'date_naissance': 'beneficiaire_date_naissance',
    'genre' : 'beneficiaire_genre',
    'allocataire-prenom': 'allocataire_prenom',
    'allocataire-nom': 'allocataire_nom',
    'beneficiaire-prenom': 'beneficiaire_prenom',
    'beneficiaire-nom': 'beneficiaire_nom'
}, inplace=True)

In [ ]:

qr_code_secret_key = os.environ['BENEF_2024_QR_CODE_URL_SECRET']
qr_code_base_url = os.environ['BENEF_2024_QR_CODE_BASE_URL']
pathfile_campaign_csv_b_and_a = os.environ['CAMPAIGN_CSV_B_AND_A']

In [ ]:
# Generate LM 
# new format for birth date
# df_campaign['beneficiaire_date_naissance'] = pd.to_datetime(df_campaign['beneficiaire_date_naissance'].str[:10], format='%Y-%m-%d')
df_campaign['beneficiaire_date_naissance'] = df_campaign['beneficiaire_date_naissance'].dt.strftime('%d/%m/%Y')

In [ ]:
# Génération des URLs pour le QR code
import urllib.parse
import base64

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

base_64_key = base64.b64decode(qr_code_secret_key)
key_mapping = { 
  'beneficiaire_prenom': 'bp', 
  'beneficiaire_nom': 'bn', 
  'beneficiaire_genre' : 'bg', 
  'beneficiaire_date_naissance': 'bdn', 
  'code': 'c'
}

def encrypt(data):
    cipher = AES.new(base_64_key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
    iv = cipher.iv
    ct = base64.b64encode(iv + ct_bytes).decode('utf-8')
    return ct

def generate_encrypted_url_column(row):
    params = {key_mapping.get(column): row[column] for column in df_campaign.columns}
    cleaned_params = {k: v for k, v in params.items() if k is not None}
    encoded_params = urllib.parse.urlencode(cleaned_params)
    encoded_encrypted_params = encrypt(encoded_params)
    full_url_string = f"{qr_code_base_url}#{urllib.parse.quote_plus(encoded_encrypted_params)}"
    return full_url_string

if 'url_qr_code' in df_campaign:
    del df_campaign['url_qr_code']

df_campaign['url_qr_code'] = df_campaign.apply(generate_encrypted_url_column, axis=1)

In [ ]:
df_campaign_new_alloc_diff_benef = df_campaign[['email', 'beneficiaire_nom', 'beneficiaire_genre', 'code', 'beneficiaire_prenom', 'allocataire_nom', 'allocataire_prenom', 'url_qr_code', 'neele', 'beneficiaire_date_naissance']]

In [ ]:
df_campaign_new_alloc_diff_benef.to_csv(pathfile_campaign_csv_b_and_a, index=False)
